# LANL EARTHQUAKE PROJECT

## packages and parameters

In [1]:
import numpy as np 
import pandas as pd
import os
%matplotlib inline

import zipfile
import pickle
import random
from scipy import stats
from scipy import fftpack

from xgboost import XGBRegressor
import seaborn as sns
import matplotlib.pyplot as plt # may remove later

from sklearn.model_selection import train_test_split
from pathlib import Path

Using TensorFlow backend.


#### set project parameters

In [1]:
# edit
PROJECT_DIR = Path('/notebooks/storage/earthquake')

NameError: name 'Path' is not defined

#### change `pwd` to `PROJECT_DIR`

In [3]:
os.chdir(PROJECT_DIR)

print(os.getcwd())
print(os.listdir())

/storage/earthquake
['test_samples.pickle', 'train.csv', 'consecutive_samples.pickle', 'submission.csv', 'model_xgb.pickle', 'refactor.ipynb', 'test_samples_eng.pickle', 'train.pickle', '.ipynb_checkpoints', 'test']


## download/unzip/load data from kaggle
- run once -> comment out
- ADD: NOTE ON WHAT HAPPENS IF IT IS RUN TWICE, REPLACEMENT OR COLLISION ????

In [ ]:
# %%time
# !kaggle competitions download LANL-Earthquake-Prediction

#### unzip kaggle files
NOTE: run once -> comment out

In [ ]:
# %%time

# train_zip='train.csv.zip'
# test_zip='test.zip'
# earthquake_dir='.'
# test_dir='./test'

# zip_ref=zipfile.ZipFile(train_zip,'r')
# zip_ref.extractall(earthquake_dir)
# zip_ref.close()

# zip_ref=zipfile.ZipFile(test_zip,'r')
# zip_ref.extractall(test_dir)
# zip_ref.close()

#### read train csv -> convert to dataframe -> pickle
NOTE: run once -> pickle `train` -> comment out

In [ ]:
%%time 

train = pd.read_csv('./train.csv', dtype={"acoustic_data": np.int16, "time_to_failure": np.float32})

#### pickle `train` dataframe
- NOTE: run once then comment out
- CAUTION !!! code below overwrites `train`!!!

In [ ]:
# %%time

# with open('/notebooks/storage/earthquake/train.pickle','wb') as f:
#     pickle.dump(train,f,pickle.HIGHEST_PROTOCOL)

#### delete `train` dataframe from ram

In [ ]:
del train

#### load `train` dataframe

In [ ]:
%%time

with open(PROJECT_DIR/'train.pickle','rb') as f:
    train = pickle.load(f)

train.tail()

## OPTIONAL: manipulate train data to exploit leakage

In [7]:
#code here

## generate consecutive samples

#### set `consecutive_sample` parameters

In [ ]:
# edit
CONS_FILENAME = 'consecutive_samples.pickle'

#leave as-is
cons_filename = PROJECT_DIR/CONS_NAME+'.pickle'

In [ ]:
def generateConsecutive(train):
    '''
    input: train dataframe (dim: #entries x 2 features: acoustic_data/time_to_failure)
    returns: dataframe with length/sample_length consecutive samples, each with:
        sequence = np.array(150k,) and time_to_failure = float
    '''
    
    samples = []
    
    length = len(train)
    sample_length = 150000
    max_index = length-1
    max_start = max_index - sample_length
    
    for i in range(length//sample_length):
        
        start = i*sample_length
        end = (i+1)*sample_length
        
        samples += [[train['acoustic_data'].values[start:end],train['time_to_failure'].values[end]]]
                
    df = pd.DataFrame(samples)
        
    df = df.rename(columns = {0:'sequence',1:'time_to_failure'})
    
    return df

In [ ]:
%%time

consecutive_samples = generateConsecutive(train)

consecutive_samples.tail()

 #### pickle `consecutive_samples`

NOTE: run once -> comment out

In [ ]:
%%time

with open(cons_filename,'wb') as f:
    pickle.dump(consecutive_samples,f,pickle.HIGHEST_PROTOCOL)

#### delete `consecutive_samples` from ram

In [ ]:
del consecutive_samples

#### load pickled `consecutive_samples` from PROJECT_DIR

In [ ]:
%%time

with open(cons_filename,'rb') as f:
    consecutive_samples = pickle.load(f)

## generate N random samples
with replacement

In [ ]:
# edit
RAND_FILENAME = 'random_samples.pickle'

#leave unchanged
rand_filename = PROJECT_DIR/CONS_NAME+'.pickle'

In [ ]:
def generateRandom(train,n):
    '''
    input: train dataframe (dim: #entries x 2 features: acoustic_data/time_to_failure)
    returns: dataframe with n random samples, each with:
        sequence = np.array(150k,) and time_to_failure = float
    '''
    
    samples = []
    
    length = len(train)
    sample_length = 150000
    max_index = length-1
    max_start = max_index - sample_length
    
    for i in range(n):
        
        start = random.randint(0,max_start)
        end = start + sample_length
        
        samples += [[np.array(train['acoustic_data'].values[start:end]),train['time_to_failure'].values[end]]]
                
    df = pd.DataFrame(samples)
        
    df = df.rename(columns = {0:'sequence',1:'time_to_failure'})
    
    return df

In [ ]:
%%time

random_samples = generateRandom(train,10)

random_samples.tail()

 #### pickle `random_samples`

NOTE: run once -> comment out

In [ ]:
%%time

with open(rand_filename,'wb') as f:
    pickle.dump(random_samples,f,pickle.HIGHEST_PROTOCOL)

#### delete `random_samples` from ram

In [ ]:
del random_samples

#### load pickled `consecutive_samples` from `PROJECT_DIR`

In [ ]:
%%time

with open(rand_filename,'rb') as f:
    random_samples = pickle.load(f)

## full sequence features
add features to a given sample

In [ ]:
def generateFullFeatures(samples_df):
    
    #defining x as matrix of sequence data
    x = []
    for index,row in samples_df.iterrows():
        x += [samples_df.loc[index,'sequence']]    
    x = np.stack(x)
    x_abs = np.absolute(x)
    
    length=150000
    
    #non-abs
    samples_df['mean'] = np.mean(x,axis=1)
    samples_df['median'] = np.median(x,axis=1)
    samples_df['std'] = np.std(x,axis=1)
    
    samples_df['kurtosis'] = stats.kurtosis(x,axis=1)   
    samples_df['m2'] = stats.moment(x,axis=1,moment=2)
    samples_df['m3'] = stats.moment(x,axis=1,moment=3)
    samples_df['skew'] = stats.skew(x,axis=1)
    samples_df['variation'] = stats.variation(x,axis=1)
    samples_df['sem'] = stats.sem(x,axis=1)

    samples_df['iqr25_75'] = stats.iqr(x,axis=1,rng=(25,75))
    samples_df['iqr10_90'] = stats.iqr(x,axis=1,rng=(10,90))
    samples_df['iqr5_95'] = stats.iqr(x,axis=1,rng=(5,95))
    samples_df['iqr1_99'] = stats.iqr(x,axis=1,rng=(1,99))
    
    
    #abs
    samples_df['mean_abs'] = np.mean(x_abs,axis=1)
    samples_df['median_abs'] = np.median(x_abs,axis=1)
    samples_df['std_abs'] = np.std(x_abs,axis=1)
    
    samples_df['kurtosis_abs'] = stats.kurtosis(x_abs,axis=1)   
    samples_df['m2_abs'] = stats.moment(x_abs,axis=1,moment=2)
    samples_df['m3_abs'] = stats.moment(x_abs,axis=1,moment=3)
    samples_df['skew_abs'] = stats.skew(x_abs,axis=1)
    samples_df['variation_abs'] = stats.variation(x_abs,axis=1)
    samples_df['sem_abs'] = stats.sem(x_abs,axis=1)

    
    samples_df['iqr25_75_abs'] = stats.iqr(x_abs,axis=1,rng=(25,75))
    samples_df['iqr10_90_abs'] = stats.iqr(x_abs,axis=1,rng=(10,90))
    samples_df['iqr5_95_abs'] = stats.iqr(x_abs,axis=1,rng=(5,95))
    samples_df['iqr1_99_abs'] = stats.iqr(x_abs,axis=1,rng=(1,99))
    
    
    #slices
    slices_list = [2,4]
    
    for slices in slices_list:

        for i in range(slices):

            suffix = '_'+str(slices)+'_'+str(i+1)

            #create same as above, but for first half and second half
            
            x_slice = x[:,i*(length//slices):(i+1)*(length//slices)]
            x_abs_slice = np.absolute(x_slice)

            #non-abs
            samples_df['mean'+suffix] = np.mean(x_slice,axis=1)
            samples_df['median'+suffix] = np.median(x_slice,axis=1)
            samples_df['std'+suffix] = np.std(x_slice,axis=1)

            samples_df['kurtosis'+suffix] = stats.kurtosis(x_slice,axis=1)
            samples_df['m2'+suffix] = stats.moment(x_slice,axis=1,moment=2)
            samples_df['m3'+suffix] = stats.moment(x_slice,axis=1,moment=3)
            samples_df['skew'+suffix] = stats.skew(x_slice,axis=1)
            samples_df['variation'+suffix] = stats.variation(x_slice,axis=1)
            samples_df['sem'+suffix] = stats.sem(x_slice,axis=1)

            samples_df['iqr25_75'+suffix] = stats.iqr(x_slice,axis=1,rng=(25,75))
            samples_df['iqr10_90'+suffix] = stats.iqr(x_slice,axis=1,rng=(10,90))
            samples_df['iqr5_95'+suffix] = stats.iqr(x_slice,axis=1,rng=(5,95))
            samples_df['iqr1_99'+suffix] = stats.iqr(x_slice,axis=1,rng=(1,99))


            #abs
            samples_df['mean_abs'+suffix] = np.mean(x_abs_slice,axis=1)
            samples_df['median_abs'+suffix] = np.median(x_abs_slice,axis=1)
            samples_df['std_abs'+suffix] = np.std(x_abs_slice,axis=1)

            samples_df['kurtosis_abs'+suffix] = stats.kurtosis(x_abs_slice,axis=1)
            samples_df['m2_abs'+suffix] = stats.moment(x_abs_slice,axis=1,moment=2)
            samples_df['m3_abs'+suffix] = stats.moment(x_abs_slice,axis=1,moment=3)
            samples_df['skew_abs'+suffix] = stats.skew(x_abs_slice,axis=1)
            samples_df['variation_abs'+suffix] = stats.variation(x_abs_slice,axis=1)
            samples_df['sem_abs'+suffix] = stats.sem(x_abs_slice,axis=1)


            samples_df['iqr25_75_abs'+suffix] = stats.iqr(x_abs_slice,axis=1,rng=(25,75))
            samples_df['iqr10_90_abs'+suffix] = stats.iqr(x_abs_slice,axis=1,rng=(10,90))
            samples_df['iqr5_95_abs'+suffix] = stats.iqr(x_abs_slice,axis=1,rng=(5,95))
            samples_df['iqr1_99_abs'+suffix] = stats.iqr(x_abs_slice,axis=1,rng=(1,99))            
            

#### generate features for a sample

In [4]:
#edit
eng_samples = random_samples
eng_samples_name = 'random_samples''

#leave as-is
eng_samples_filename = PROJECT_DIR/eng_samples_name+'.pickle'

NameError: name 'random_samples' is not defined

In [ ]:
%%time

generateFullFeatures(eng_samples)

eng_samples.tail()

 #### pickle `eng_samples`

NOTE: run once -> comment out

In [ ]:
%%time

with open(eng_samples_filename,'wb') as f:
    pickle.dump(eng_samples,f,pickle.HIGHEST_PROTOCOL)

#### delete `eng_samples` from ram

In [5]:
del eng_samples

#### load pickled `eng_samples` from PROJECT_DIR

In [ ]:
%%time

with open(eng_samples_filename,'rb') as f:
    eng_samples = pickle.load(f)

## visualize data

In [ ]:
random_samples.plot(x='iqr1_99_abs',y='time_to_failure',kind='scatter',logx=True)

In [ ]:
random_samples.plot.hexbin(x='mean',y='time_to_failure',gridsize=10)

In [ ]:
sns.kdeplot(consecutive_samples['time_to_failure'],consecutive_samples['mean'])

In [ ]:
sns.kdeplot(consecutive_samples.time_to_failure)


In [ ]:
sns.jointplot(x='iqr1_99_abs',y='time_to_failure',data=consecutive_samples,kind='hex',gridsize=10)

In [ ]:
sns.pairplot(consecutive_samples[['time_to_failure','iqr1_99_abs','iqr5_95_abs','iqr10_90_abs']])

In [ ]:
# look into ggplot for grammar of graphics

## split data into train/validation

In [ ]:
# public_mean = 4.017 
# train_mean = 5.68
# mul_factor = public_mean/train_mean

X = random_samples.drop(labels=['time_to_failure','sequence'],axis=1)
Y = random_samples['time_to_failure']/mul_factor

x_train, x_val, y_train, y_val = train_test_split(X,Y,test_size=.1,random_state=0,shuffle=False)

x_val.shape

## XGBoost

In [ ]:
n_estimators = 10000
learning_rate = .001
n_jobs = 8

early_stopping_rounds = 20
eval_set = [(x_val,y_val)]
verbose = True

In [ ]:
%%time

model = XGBRegressor(
    n_estimators = n_estimators, 
    learning_rate = learning_rate, 
    n_job = n_jobs
)

model.fit(x_train,y_train,
              early_stopping_rounds= early_stopping_rounds,
              eval_set = eval_set,
              eval_metric = 'mae',
              verbose = verbose
             )

from sklearn.metrics import mean_absolute_error


predictions = model.predict(x_val)
print('mae : '+str(mean_absolute_error(predictions,y_val)))

#### pickle `model`

In [ ]:
# edit
MODEL_NAME = 'model1.pickle'

#leave as-is
model_filename = PROJECT_DIR/MODEL_NAME+'.pickle'

In [ ]:
%%time

with open(model_filename,'wb') as f:
    pickle.dump(model,f,pickle.HIGHEST_PROTOCOL)

#### delete `model` from environment

In [ ]:
del model

#### load pickled `model`

In [ ]:
%%time

with open(model_filename,'rb') as f:
    model = pickle.load(f)

## create submission
#### start by creating a dataframe identical in format to consecutive/random samples above

In [ ]:
# edit
TEST_SAMPLES_NAME = 'test_samples'

# leave as-is
test_samples_filename = PROJECT_DIR/'test'/TEST_SAMPLES_NAME+'.pickle'

In [ ]:
%%time

os.chdir(PROJECT_DIR/'test')
    
test_samples = pd.DataFrame(columns=['sequence','time_to_failure'])

for i in range(len(os.listdir())):
    
    test_file = os.listdir()[i]
    
    temp_df = pd.read_csv('./'+test_file,engine='python')
        
    test_samples.loc[i,'sequence'] = np.array(temp_df['acoustic_data'].values[:])

test_samples.tail()

#### pickle `test_samples`

In [ ]:
%%time

with open(test_samples_filename,'wb') as f:
    pickle.dump(test_samples,f,pickle.HIGHEST_PROTOCOL)

#### delete `test_samples` from environment

In [ ]:
del test_samples

#### load `test_samples`

In [ ]:
%%time

with open(test_samples_filename,'rb') as f:
    test_samples = pickle.load(f)

## add features

In [ ]:
%time

generateFullFeatures(test_samples)

test_samples.tail()

#### pickle `test_samples_eng`

In [ ]:
# %%time

# with open(PROJECT_DIR/'test'/TEST_SAMPLES_NAME+'_eng','wb') as f:
#     pickle.dump(test_samples,f,pickle.HIGHEST_PROTOCOL)

#### delete `test_samples_eng` from environment

In [ ]:
del test_samples

#### load `test_samples_eng`

In [ ]:
%%time

with open(PROJECT_DIR/'test'/TEST_SAMPLES_NAME+'_eng','rb') as f:
    test_samples = pickle.load(f)

test_samples.shape

## make predictions/submit
This code will create a `submission.csv` file, that can be submitted to kaggle using the command line api, or website

In [ ]:
test_x = test_samples.drop(columns=['sequence','time_to_failure'])

y_pred = model_xgb.predict(test_x)

y_pred.shape

submission_df = pd.DataFrame(columns=['seg_id','time_to_failure'])

submission_df['seg_id'] = pd.Series([i[:-4] for i in os.listdir(PROJECT_DIR/'test')])
submission_df.shape

submission_df['time_to_failure'] = y_pred

submission_df.tail()

submission_df.to_csv(PROJECT_DIR/'submission.csv',index=False)